**Table of contents**<a id='toc0_'></a>    
- [Mise en place](#toc1_)    
  - [Chargement des données](#toc1_1_)    
- [🦄🦄 CHECKPOINT 🦄🦄](#toc2_)    
- [🚧 TODO](#toc3_)    
- [TF-IDF](#toc4_)    
- [FYI 🚧](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=3
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Mise en place](#toc0_)

In [23]:
# OS & env
import os
import logging

# DS
import numpy as np
import pandas as pd
import dill as pickle

# ML
from sklearn.feature_extraction.text import TfidfVectorizer

# home made functions from the src folder
from src.scrap_and_clean import words_filter

# logging configuration (see all outputs, even DEBUG or INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## <a id='toc1_1_'></a>[Chargement des données](#toc0_)

In [24]:
if not os.path.exists("data/df_preprocessed.pkl") or not os.path.exists(
    "data/stats_df.pkl"
):
    logging.warning("Missing data: run EDA notebook first")
else:
    with open("data/df_preprocessed.pkl", "rb") as df:
        df = pickle.load(df)
    with open("data/stats_df.pkl", "rb") as st:
        stats_df = pickle.load(st)

Visualisation des données :

In [25]:
print(f"Data shape: {df.shape}")
display(df.head())
print(f'Corpus contains {len(stats_df["count_corpus"])} tokens')
display(stats_df.loc[stats_df.token == "c++"])

Data shape: (49992, 9)


,Title,Body,Tags,Score,AnswerCount,CreationDate,ViewCount,title_tokens,body_tokens
0,itms- : missing api declaration - privacy,why am i all of a suddent getting this on succ...,"[ios, app-store, plist, appstore-approval, pri...",24,7,2024-03-14 22:55:18,3092,"[itms, missing, api, declaration, privacy]","[suddent, getting, successful, builds, apple]"
1,why is builtin sorted() slower for a list cont...,i sorted four similar lists. list consistently...,"[python, algorithm, performance, sorting, time...",28,2,2024-03-05 15:21:50,2699,"[builtin, sorted, slower, list, containing, de...","[sorted, four, similar, lists, list, consisten..."
2,std::shared_mutex::unlock_shared() blocks even...,my team has encountered a deadlock that i susp...,"[c++, windows, multithreading, stl, shared-lock]",26,5,2024-03-01 23:09:59,1388,"[std, shared_mutex, unlock_shared, blocks, eve...","[team, encountered, deadlock, suspect, bug, wi..."
3,did the rules for nullptr init of unique_ptr c...,this code compiles with msvc from vs in c++ mo...,"[c++, visual-c++, language-lawyer, unique-ptr,...",15,1,2024-02-22 11:29:42,490,"[rules, nullptr, init, unique_ptr, change, c++]","[code, compiles, msvc, c++, mode, failes, c++,..."
4,where is the order in which elf relocations ar...,consider the following two files on a linux sy...,"[c++, elf, dynamic-linking, abi, relocation]",16,1,2024-02-19 21:42:03,1746,"[order, elf, relocations, applied, specified]","[consider, following, two, files, linux, syste..."


Corpus contains 110214 tokens


,token,count_corpus,freq_corpus,count_tags,freq_tags,count_title,freq_title,count_body,freq_body
15268,c++,3645,0.001073,3115,0.012462,716,0.002324,2929,0.000949


In [26]:
display(stats_df)

,token,count_corpus,freq_corpus,count_tags,freq_tags,count_title,freq_title,count_body,freq_body
0,.a,28,8.245222e-06,1,0.000004,6,0.000019,22,7.124865e-06
1,.d.ts,18,5.300500e-06,4,0.000016,1,0.000003,17,5.505577e-06
2,.extend.withvalue,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
3,.htaccess,55,1.619597e-05,54,0.000216,7,0.000023,48,1.554516e-05
4,.net,3089,9.096247e-04,1277,0.005109,638,0.002071,2451,7.937747e-04
...,...,...,...,...,...,...,...,...,...
110209,，it,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
110210,，when,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
110211,�pl+�li���d��,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
110212,�s˸8ƍ�,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07


In [27]:
stats_df.sort_values(by="count_corpus", ascending=False)

,token,count_corpus,freq_corpus,count_tags,freq_tags,count_title,freq_title,count_body,freq_body
103005,using,37631,1.108128e-02,8,0.000032,4043,0.013121,33588,1.087772e-02
18727,code,27900,8.215775e-03,0,0.000000,982,0.003187,26918,8.717596e-03
83694,s,26626,7.840618e-03,0,0.000000,1265,0.004105,25361,8.213350e-03
53641,like,25259,7.438074e-03,0,0.000000,244,0.000792,25015,8.101295e-03
102389,use,24975,7.354444e-03,0,0.000000,2108,0.006841,22867,7.405649e-03
...,...,...,...,...,...,...,...,...,...
59293,mkl_set_num_cores,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
59294,mklink,1,2.944722e-07,0,0.000000,0,0.000000,1,3.238575e-07
59295,mklocalsearchcompleter,1,2.944722e-07,0,0.000000,1,0.000003,0,0.000000e+00
17603,circuiting,1,2.944722e-07,0,0.000000,1,0.000003,0,0.000000e+00


# <a id='toc2_'></a>[🦄🦄 CHECKPOINT 🦄🦄](#toc0_)

# <a id='toc3_'></a>[🚧 TODO](#toc0_)

- aller sur les modèles (non supervisé puis supervisé puis word2vec, cf. ci-dessous)
- voir NMF et LDA et leurs différences, points forts et usages
- non supervisé : faire un premier algo simple (régression logistique, SGD ... et voir comment faire pour deviner plusieurs labels (regarder avec predict_proba aussi)
- faire des prédictions
- aller jusqu’au bout de l’exercice et voir après
- regarder si des classes de mots (par exemple les adjectifs) sont à enlever à un moment pour améliorer les capacités

# <a id='toc4_'></a>[TF-IDF](#toc0_)

In [28]:
corpus = df["title_tokens"].apply(lambda x: " ".join(x)).to_list()

vectorizer = TfidfVectorizer(token_pattern=r"\S+")
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

array(['.a', '.d.ts', '.htaccess', ..., '≥3gb', 'ꭲᏻꮎꮅꮝꮤꮕ', 'ꮝꭶꮪꭹ'],
      dtype=object)

🚧 retrouver quelle est la valeur à choper : la plus grande ou la plus petite ?

In [29]:
print(vectorizer.get_feature_names_out()[27374])
print(vectorizer.get_feature_names_out()[5850])

zero-allocation
deducing


In [30]:
print(X.shape)
print(corpus[3])
print(X[3])
print(corpus[6])
print(X[6])
print(corpus[7])
print(X[7])

(49992, 27467)
rules nullptr init unique_ptr change c++
  (0, 3130)	0.2834497346141111
  (0, 3539)	0.2899654868517161
  (0, 25379)	0.44353148817255206
  (0, 11024)	0.4337465763016716
  (0, 16103)	0.5227410752336996
  (0, 20711)	0.4212621027241501
possible make zero-allocation coroutine runtime c++
  (0, 20733)	0.38323367323801016
  (0, 4957)	0.44759039077813456
  (0, 27374)	0.6106012392508993
  (0, 13599)	0.30367593187981645
  (0, 18061)	0.322079623372116
  (0, 3130)	0.28983886042117574
combined c++ deducing conversion operator auto return type
  (0, 24937)	0.23851073947605095
  (0, 20389)	0.28451267433583277
  (0, 1961)	0.33663611732903503
  (0, 16656)	0.32927354874944426
  (0, 4853)	0.3606642271063403
  (0, 5850)	0.5117041064725818
  (0, 4271)	0.4259144731422853
  (0, 3130)	0.25903147332563975


In [31]:
_ = X[3].copy()
d = X[3].copy().data
i = X[3].copy().indices

print(d)
print(i)
print(_)

sorted_x = np.argsort(d)
print(sorted_x)
sorted_x = np.argsort(-d)
print(sorted_x)


def predict_idf(vectorizer, X, n_max):
    """Predict a maximum of n_max results from a vectorizer and a transformed sparse matrix X."""
    d = X.data
    i = X.indices

    # get highest weights
    weights = [np.sort(d)[::-1][: min(len(i), n_max)]]

    # get corresponding indices
    pred_indices = [i[np.argsort(-d)][: min(len(i), n_max)]]

    # get corresponding words
    preds = [[vectorizer.get_feature_names_out()[x] for x in pred_indices[0]]]

    return preds, weights


print(i[np.argsort(-d)][: min(len(i), 3)])

print(corpus[3])
predict_idf(vectorizer, X[3], 3)


# def predict_idf(vectorizer, X, n_max):
#     results = []
#     for i, e in enumerate(X.data):
#         results.append(vectorizer.get_feature_names_out()[x])
#     return results

# for x in i:
#     results.append(vectorizer.get_feature_names_out()[x])
# results

[0.28344973 0.28996549 0.44353149 0.43374658 0.52274108 0.4212621 ]
[ 3130  3539 25379 11024 16103 20711]
  (0, 3130)	0.2834497346141111
  (0, 3539)	0.2899654868517161
  (0, 25379)	0.44353148817255206
  (0, 11024)	0.4337465763016716
  (0, 16103)	0.5227410752336996
  (0, 20711)	0.4212621027241501
[0 1 5 3 2 4]
[4 2 3 5 1 0]
[16103 25379 11024]
rules nullptr init unique_ptr change c++


([['nullptr', 'unique_ptr', 'init']],
 [array([0.52274108, 0.44353149, 0.43374658])])

In [32]:
print(X.shape)

(49992, 27467)


# <a id='toc5_'></a>[FYI 🚧](#toc0_)

🚧 add / remove item

In [33]:
# items_to_keep = ["s", "t", "d"]
# keep_set, exclude_set = words_filter(items_to_keep, "add", keep_set, exclude_set)

# items_to_remove = [s for s in mutual if s not in items_to_keep]
# keep_set, exclude_set = words_filter(items_to_remove, "rm", keep_set, exclude_set)